In [1]:
#Importing required modules
from lxml import objectify
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import nltk
from nltk.corpus import stopwords
import string

In [2]:
#Opening the TASS 2015 CSV file
tassdb = pd.read_csv('TASS_tweet_database.csv', encoding='latin1')

In [3]:
tassdb.head()

,tweetid,user,content,date,lang,value,topic
0,1.423780e+17,jesusmarana,"Portada 'Público', viernes. Fabra al banquillo...",02/12/2011 0:03,es,N,política
1,1.423790e+17,EvaORegan,"Grande! RT @veronicacalderon ""El periodista es...",02/12/2011 0:06,es,NONE,política
2,1.423790e+17,LosadaPescador,Gonzalo Altozano tras la presentación de su li...,02/12/2011 0:06,es,P+,otros
3,1.423800e+17,mgilguerrero,"Mañana en Gaceta: TVE, la que pagamos tú y yo,...",02/12/2011 0:09,es,N,entretenimiento
4,1.423810e+17,pedroj_ramirez,Qué envidia @mfcastineiras: Pedro mañana x la...,02/12/2011 0:14,es,NONE,otros


In [4]:
#getting the root node from the XML file
tassdb.content[0]

u"Portada 'P\xfablico', viernes. Fabra al banquillo por 'orden' del Supremo; Wikileaks 'retrata' a 160 empresas esp\xedas. http://t.co/YtpRU0fd"

In [5]:
tassdb = (tassdb[tassdb.value != 'NONE'])

In [6]:
len(tassdb)

45396

In [7]:
tassdb.head()

,tweetid,user,content,date,lang,value,topic
0,1.423780e+17,jesusmarana,"Portada 'Público', viernes. Fabra al banquillo...",02/12/2011 0:03,es,N,política
2,1.423790e+17,LosadaPescador,Gonzalo Altozano tras la presentación de su li...,02/12/2011 0:06,es,P+,otros
3,1.423800e+17,mgilguerrero,"Mañana en Gaceta: TVE, la que pagamos tú y yo,...",02/12/2011 0:09,es,N,entretenimiento
5,1.423830e+17,mgilguerrero,Más mañana en Gaceta. Amaiur depende de Uxue B...,02/12/2011 0:20,es,N,política
6,1.423830e+17,SSantiagosegura,"Muy buenas noches followercetes, mañana va a s...",02/12/2011 0:20,es,P+,otros


In [8]:
tassdb.value.value_counts()

P+     23545
N      12839
N+      5575
P       1939
NEU     1498
Name: value, dtype: int64

In [9]:
values = {'P+':1, 'P':0.5, 'NEU':0, 'N': -0.5, 'N+':-1 }

In [10]:
tassdb.value.replace(values, inplace=True)

In [11]:
tassdb.head()

,tweetid,user,content,date,lang,value,topic
0,1.423780e+17,jesusmarana,"Portada 'Público', viernes. Fabra al banquillo...",02/12/2011 0:03,es,-0.5,política
2,1.423790e+17,LosadaPescador,Gonzalo Altozano tras la presentación de su li...,02/12/2011 0:06,es,1.0,otros
3,1.423800e+17,mgilguerrero,"Mañana en Gaceta: TVE, la que pagamos tú y yo,...",02/12/2011 0:09,es,-0.5,entretenimiento
5,1.423830e+17,mgilguerrero,Más mañana en Gaceta. Amaiur depende de Uxue B...,02/12/2011 0:20,es,-0.5,política
6,1.423830e+17,SSantiagosegura,"Muy buenas noches followercetes, mañana va a s...",02/12/2011 0:20,es,1.0,otros


In [12]:
tassdb.value.value_counts()

 1.0    23545
-0.5    12839
-1.0     5575
 0.5     1939
 0.0     1498
Name: value, dtype: int64

In [13]:
#First step to standarize the text data is to unifiy all text into lower cases
tassdb.content = tassdb.content.str.lower()

In [14]:
tassdb.content = tassdb.content.str.replace('/ ' , '/')

In [15]:
tassdb.content = tassdb.content.str.replace('w. ', 'w.')

In [16]:
tassdb = tassdb.reset_index()
del(tassdb['index']) 

In [17]:
tassdb.head()

,tweetid,user,content,date,lang,value,topic
0,1.423780e+17,jesusmarana,"portada 'público', viernes. fabra al banquillo...",02/12/2011 0:03,es,-0.5,política
1,1.423790e+17,LosadaPescador,gonzalo altozano tras la presentación de su li...,02/12/2011 0:06,es,1.0,otros
2,1.423800e+17,mgilguerrero,"mañana en gaceta: tve, la que pagamos tú y yo,...",02/12/2011 0:09,es,-0.5,entretenimiento
3,1.423830e+17,mgilguerrero,más mañana en gaceta. amaiur depende de uxue b...,02/12/2011 0:20,es,-0.5,política
4,1.423830e+17,SSantiagosegura,"muy buenas noches followercetes, mañana va a s...",02/12/2011 0:20,es,1.0,otros


In [18]:
tassdb.content[0]

u"portada 'p\xfablico', viernes. fabra al banquillo por 'orden' del supremo; wikileaks 'retrata' a 160 empresas esp\xedas. http://t.co/ytpru0fd"

In [19]:
#Then from the strings library we import the punctuation list
#and from nklt the stop words list in this case for spanish
#I decided to add some aadditional terms that I want to remove in order to obatin a cleaner
#list of the most used words
punctuation = list(string.punctuation)
stop = stopwords.words('spanish') + punctuation + ['rt', 'via', u'', u'\u2026,', u'\u2026', u'...', u'...,']

In [120]:
#To obtain the actual text for each tweet we split each word
tasssplit = tassdb.content.str.split(' ')

In [130]:
tasstweetlist = []
for i in range (0, len(tasssplit)):
    a= [term.rstrip(''.join(punctuation)) for term in tasssplit[i]]
    tasstweetlist.append(a)

In [132]:
tasstweetlist1 = []
for i in range (0, len(tasstweetlist)):
    tasstweetlist1.append([term for term in tasstweetlist[i] if term not in stop])

In [134]:
for lists in tasstweetlist1:
    for words in lists:
        if words.startswith('http:') == True or words.startswith('@') == True:
            lists.remove(words) 

In [138]:
print(tasstweetlist1[200])

[u'fuentes', u'gobierno', u'confirman', u'reglamento', u'ley', u'sinde', u'tampoco', u'aprobar\xe1', u'pr\xf3xima', u'semana']


In [ ]:
tasstweetlistws = []
for i in range (0, len(tasstweetlist)):
    a = " ".join(tasstweetlist[i])
    tasstweetlistws.append(a)

In [ ]:
tasstweetlistws

In [ ]:
punctuation